In [1]:
import pandas as pd
import numpy as np
import glob
import os
import csv
import sys
import seaborn as sns
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm
from scipy.stats import linregress
#import plotly.graph_objects as go
#from plotly.subplots import make_subplots


sns.set_style('darkgrid')

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib

#setting pandas display options
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199

BASE_DIR = "/Users/karinalopez/Desktop/ds_projects/hack4la_airbnb/"

<ipython-input-1-65c36a9b7c00>:26: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)  # or 199


In [3]:
os.chdir(BASE_DIR + 'data/raw/airbnb/feb_2021/')

list_feb_2021_df = pd.read_csv('listings.csv')
rev_feb_2021_df = pd.read_csv('reviews.csv')

In [5]:
print(list_feb_2021_df.shape)


(32175, 74)


Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_upd

In [7]:
col = list_feb_2021_df.columns

In [8]:
for i in col:
    print(i)

id
listing_url
scrape_id
last_scraped
name
description
neighborhood_overview
picture_url
host_id
host_url
host_name
host_since
host_location
host_about
host_response_time
host_response_rate
host_acceptance_rate
host_is_superhost
host_thumbnail_url
host_picture_url
host_neighbourhood
host_listings_count
host_total_listings_count
host_verifications
host_has_profile_pic
host_identity_verified
neighbourhood
neighbourhood_cleansed
neighbourhood_group_cleansed
latitude
longitude
property_type
room_type
accommodates
bathrooms
bathrooms_text
bedrooms
beds
amenities
price
minimum_nights
maximum_nights
minimum_minimum_nights
maximum_minimum_nights
minimum_maximum_nights
maximum_maximum_nights
minimum_nights_avg_ntm
maximum_nights_avg_ntm
calendar_updated
has_availability
availability_30
availability_60
availability_90
availability_365
calendar_last_scraped
number_of_reviews
number_of_reviews_ltm
number_of_reviews_l30d
first_review
last_review
review_scores_rating
review_scores_accuracy
review_sc

In [6]:
print(rev_feb_2021_df.shape)
rev_feb_2021_df.columns

(1106072, 6)


Index(['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments'], dtype='object')

In [9]:
col = rev_feb_2021_df.columns
for i in col:
    print(i)

listing_id
id
date
reviewer_id
reviewer_name
comments


In [21]:
rev_feb_2021_df.id.nunique()

1106072

In [20]:
list_feb_2021_df.id.nunique()

32175

In [12]:
list_feb_2021_df.head()

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,109,https://www.airbnb.com/rooms/109,20210102020546,2021-01-05,Amazing bright elegant condo park front *UPGRADED*,"*** Unit upgraded with new bamboo flooring, brand new Ultra HD 50"" Sony TV, new paint, new lighting, new mattresses, ultra fast cable Internet connection, Apple TV, Google Chromecast. ***<br /><br />Gorgeous and Elegant Furnished Condo in front of Culver City Fox Hills Park. <br />Upper corner unit, total silence protected by trees.<br />Short walk to the new Westfield Mall.<br />Tennis courts, heated pool and jacuzzi hot tub.<br /><br /><b>The space</b><br />*** Unit upgraded with new bamboo flooring, brand new Ultra HD 50"" Sony TV, new paint, new lighting, new mattresses, ultra fast cable Internet connection. ***<br /><br />Gorgeous and Elegant Furnished Apartment in front of Culver City Fox Hills Park. <br />Upper corner unit, total silence protected by trees.<br />Short walk to the new Westfield Mall.<br />Tennis courts, heated pool and jacuzzi hot tub.<br /><br />*** Upgraded with bamboo flooring and new paint the whole apartment ***<br />Just installed gorgeous high quality bambo",NaN,https://a0.muscache.com/pictures/4321499/1da9892a_original.jpg,521,https://www.airbnb.com/users/show/521,Paolo,2008-06-27,"San Francisco, California, United States","Paolo Privitera, CEO Evensi\n\nPaolo, MIT MBA (Phone number hidden by Airbnb) , is currently the CEO and co-founder of Evensi, the world's largest events discovery and promotion network, with more than 200 million events, 50 million users, 10 million event organizers. The company has been approved for an IPO.\n\nPaolo is a Silicon Valley-based global entrepreneur and strategist with a passion for innovation and growth using the power of data, strategic analysis, creativity, and teamwork.\nBorn and raised in Venice Italy, he founded his first Internet company in (Phone number hidden by Airbnb) when he was 16 years old.\nPaolo has been a U.S. resident since (Phone number hidden by Airbnb) He has more than 25 years of experience in tech and internet-based businesses. During this time, he has founded from scratch 6 companies, invested in more than 50, and advised hundreds. Paolo has received 10 career awards and 4 of the companies he founded had an exit. \n\nHe is also a mentor and advisor at 500 Startups, (Hidden by Airbnb) Launchpad, China Accelerator, Alchemist, Start-Up Chile, Health Wildcatters, Nana Bianca, The Vault Korea, Galvanize, and Future Food Institute. \n\nHe loves to give back and help other people and companies.\n\n- Search for me on the Internet with the keyword ""pppaolo""\n- I am close friend with Airbnb founder since (Phone number hidden by Airbnb) , a

In [19]:
# chcek 'Ankit' exist in dataframe or not
if 721123480 in list_feb_2021_df['id'].values :
    print("\nThis value exists in Dataframe")
  
else:
    print("\nThis value does not exists in Dataframe")


This value does not exists in Dataframe


In [22]:
list_id = list_feb_2021_df.id.unique()

In [24]:
temp = list_id[:25]

In [28]:
count = 0
for i in list_id:
    # chcek 'Ankit' exist in dataframe or not
    if i in rev_feb_2021_df['id'].values :
        print(str(i) + "\nThis value exists in Dataframe")
        count += 1


1083801
This value exists in Dataframe
4233266
This value exists in Dataframe
13867920
This value exists in Dataframe
14640082
This value exists in Dataframe
17622948
This value exists in Dataframe
17792460
This value exists in Dataframe
17997198
This value exists in Dataframe
19719971
This value exists in Dataframe
19908669
This value exists in Dataframe
20173935
This value exists in Dataframe
21679388
This value exists in Dataframe
23991456
This value exists in Dataframe
24160098
This value exists in Dataframe
25347310
This value exists in Dataframe
27707551
This value exists in Dataframe
28408964
This value exists in Dataframe
28766699
This value exists in Dataframe
29192160
This value exists in Dataframe
29309098
This value exists in Dataframe
30350595
This value exists in Dataframe
34842295
This value exists in Dataframe
35597003
This value exists in Dataframe
36231731
This value exists in Dataframe
36644699
This value exists in Dataframe
39226551
This value exists in Dataframe
39